<a href="https://colab.research.google.com/github/ailab-nda/ML/blob/main/sbt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Soul Bound Token (SBT) の python 実装

In [ ]:
import hashlib
import datetime
import json
from typing import Dict, Optional

## SBT の定義

In [ ]:
class SoulBoundToken:
    """
    Represents a simplified SoulBound Token (SBT).  This is a conceptual
    implementation and does NOT interact with a real blockchain. It demonstrates
    the core principles of non-transferability and basic data storage.

    Attributes:
        issuer (str):  The identifier of the entity issuing the token.
        recipient (str): The identifier of the entity receiving the token (e.g., a wallet address).
        data (dict):  A dictionary containing the token's data (e.g., credentials, achievements).
        timestamp (datetime): The time the token was created.
        token_id (str):  A unique identifier for the token (hash).
        revoked (bool): Indicates if the token has been revoked by the issuer.
        revocation_reason (Optional[str]):  The reason for revocation, if applicable.
        previous_token_id (Optional[str]):  For updates/versions, links to the previous token.

    """
    def __init__(self, issuer: str, recipient: str, data: Dict, previous_token_id: Optional[str] = None):
        self.issuer = issuer
        self.recipient = recipient
        self.data = data
        self.timestamp = datetime.datetime.utcnow()
        self.previous_token_id = previous_token_id  # Initialize *before* using it!
        self.token_id = self._generate_token_id()
        self.revoked = False
        self.revocation_reason = None


    def _generate_token_id(self) -> str:
        """Generates a unique token ID using a hash."""
        data_string = f"{self.issuer}{self.recipient}{self.data}{self.timestamp}{self.previous_token_id}".encode('utf-8')
        return hashlib.sha256(data_string).hexdigest()

    def revoke(self, reason: str, revoker: str):
        """Revokes the token.  Only the issuer can revoke."""
        if revoker == self.issuer:
            self.revoked = True
            self.revocation_reason = reason
        else:
            raise ValueError("Only the issuer can revoke the token.")

    def update_data(self, new_data: Dict, updater:str) -> "SoulBoundToken":
      """
      Creates a *new* SBT with updated data, linking it to the previous one.
      This simulates an update, as the original SBT cannot be modified.

      Args:
          new_data: The updated data.
          updater: The ID of who is performing the update (must be issuer or recipient).

      Returns:
          A new SoulBoundToken instance representing the updated version.

      Raises:
          ValueError:  If the updater is not the issuer or recipient
      """
      if updater != self.issuer and updater != self.recipient:
        raise ValueError("Only the issuer or recipient can update the token data.")

      return SoulBoundToken(self.issuer, self.recipient, new_data, previous_token_id=self.token_id)


    def to_dict(self) -> Dict:
        """Converts the token to a dictionary for serialization."""
        return {
            "token_id": self.token_id,
            "issuer": self.issuer,
            "recipient": self.recipient,
            "data": self.data,
            "timestamp": self.timestamp.isoformat(),
            "revoked": self.revoked,
            "revocation_reason": self.revocation_reason,
            "previous_token_id": self.previous_token_id
        }

    def __str__(self):
        return json.dumps(self.to_dict(), indent=4)

    def is_valid(self):
        """Basic validity check (not a full blockchain validation)."""
        # Check if the token ID is correct based on the current data.
        expected_token_id = self._generate_token_id()
        if self.token_id != expected_token_id:
            return False

        # Check if the token is revoked.
        if self.revoked:
            return False

        # Add more checks as needed for your specific use case
        # (e.g., data schema validation).
        return True

## 使用例

### 1. トークン発行

In [ ]:
issuer_id = "University of Example"
recipient_id = "0xabcd1234walletAddress"  # Example recipient identifier
credential_data = {
    "degree": "Bachelor of Science",
    "major": "Computer Science",
    "graduation_date": "2024-05-15",
}

sbt = SoulBoundToken(issuer_id, recipient_id, credential_data)
print("--- Issued SBT ---")
print(sbt)
print(f"Is valid: {sbt.is_valid()}")

--- Issued SBT ---
{
    "token_id": "8dd1b9701ac5952267d7ee38cdefe3c0920cc8fa6d8a38d4d80bda6f692b6b34",
    "issuer": "University of Example",
    "recipient": "0xabcd1234walletAddress",
    "data": {
        "degree": "Bachelor of Science",
        "major": "Computer Science",
        "graduation_date": "2024-05-15"
    },
    "timestamp": "2025-02-26T08:52:31.449564",
    "revoked": false,
    "revocation_reason": null,
    "previous_token_id": null
}
Is valid: True


/var/folders/f5/4kxhzrrd5_16jg94_dpsjq5m0000gn/T/ipykernel_56165/3759532336.py:22: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  self.timestamp = datetime.datetime.utcnow()


### 2. トークンの移動

In [ ]:
# トークンは移動できない（移動させる関数を作成してはいけない）

### 3. トークンの廃止

In [ ]:
try:
    sbt.revoke("Incorrect major listed", "SomeOtherEntity")  # Should fail
except ValueError as e:
    print(f"\nError during revocation (expected): {e}")

sbt.revoke("Incorrect major listed", issuer_id)
print("\n--- Revoked SBT ---")
print(sbt)
print(f"Is valid: {sbt.is_valid()}")


Error during revocation (expected): Only the issuer can revoke the token.

--- Revoked SBT ---
{
    "token_id": "8dd1b9701ac5952267d7ee38cdefe3c0920cc8fa6d8a38d4d80bda6f692b6b34",
    "issuer": "University of Example",
    "recipient": "0xabcd1234walletAddress",
    "data": {
        "degree": "Bachelor of Science",
        "major": "Computer Science",
        "graduation_date": "2024-05-15"
    },
    "timestamp": "2025-02-26T08:52:31.449564",
    "revoked": true,
    "revocation_reason": "Incorrect major listed",
    "previous_token_id": null
}
Is valid: False


### 4. データの更新（新しくリンクされたトークンを作成）

In [ ]:
updated_data = {
    "degree": "Bachelor of Science",
    "major": "Computer Engineering",  # Corrected major
    "graduation_date": "2024-05-15",
}

updated_sbt = sbt.update_data(updated_data, issuer_id) # Issuer updates
print("\n--- Updated SBT ---")
print(updated_sbt)
print(f"Is valid (updated): {updated_sbt.is_valid()}")
print(f"Is valid (original): {sbt.is_valid()}") #Original is still revoked (and invalid)

#Recipient Updates
updated_data2 = {
    "degree": "Bachelor of Science",
    "major": "Computer Engineering",
    "graduation_date": "2024-05-15",
    "honors": "Cum Laude"
}

updated_sbt2 = updated_sbt.update_data(updated_data2, recipient_id) #Recipient updates
print("\n--- Updated SBT by Recipient ---")
print(updated_sbt2)

#Try to update from someone other than issuer/recipient.

try:
    updated_sbt3 = updated_sbt2.update_data(updated_data2, "0xSomeOtherAddress") #Recipient updates
except ValueError as e:
    print(f"\nError during update (expected): {e}")


--- Updated SBT ---
{
    "token_id": "52a7e40beb0b17d5261d1b370d5cca376e4b524e45891e058c2df29aa54e4da8",
    "issuer": "University of Example",
    "recipient": "0xabcd1234walletAddress",
    "data": {
        "degree": "Bachelor of Science",
        "major": "Computer Engineering",
        "graduation_date": "2024-05-15"
    },
    "timestamp": "2025-02-26T08:53:32.404020",
    "revoked": false,
    "revocation_reason": null,
    "previous_token_id": "8dd1b9701ac5952267d7ee38cdefe3c0920cc8fa6d8a38d4d80bda6f692b6b34"
}
Is valid (updated): True
Is valid (original): False

--- Updated SBT by Recipient ---
{
    "token_id": "b4fb042ccfb2a0f912df0ee232a2df6f9d28fb25a77b21f7700bc8340a6fe318",
    "issuer": "University of Example",
    "recipient": "0xabcd1234walletAddress",
    "data": {
        "degree": "Bachelor of Science",
        "major": "Computer Engineering",
        "graduation_date": "2024-05-15",
        "honors": "Cum Laude"
    },
    "timestamp": "2025-02-26T08:53:32.404396

/var/folders/f5/4kxhzrrd5_16jg94_dpsjq5m0000gn/T/ipykernel_56165/3759532336.py:22: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  self.timestamp = datetime.datetime.utcnow()


### 5. シリアライズ

In [ ]:
sbt_json = json.dumps(sbt.to_dict())
print("\n--- Serialized SBT (JSON) ---")
print(sbt_json)


--- Serialized SBT (JSON) ---
{"token_id": "8dd1b9701ac5952267d7ee38cdefe3c0920cc8fa6d8a38d4d80bda6f692b6b34", "issuer": "University of Example", "recipient": "0xabcd1234walletAddress", "data": {"degree": "Bachelor of Science", "major": "Computer Science", "graduation_date": "2024-05-15"}, "timestamp": "2025-02-26T08:52:31.449564", "revoked": true, "revocation_reason": "Incorrect major listed", "previous_token_id": null}


### 6. シリアライズ解除

In [ ]:
loaded_sbt_data = json.loads(sbt_json)
loaded_sbt = SoulBoundToken(
    loaded_sbt_data["issuer"],
    loaded_sbt_data["recipient"],
    loaded_sbt_data["data"],
     #previous_token_id must be explicitly handled.
)
loaded_sbt.timestamp = datetime.datetime.fromisoformat(loaded_sbt_data["timestamp"])
loaded_sbt.token_id = loaded_sbt_data["token_id"]
loaded_sbt.revoked = loaded_sbt_data["revoked"]
loaded_sbt.revocation_reason = loaded_sbt_data["revocation_reason"]

print("\n--- Deserialized SBT ---")
print(loaded_sbt)


--- Deserialized SBT ---
{
    "token_id": "8dd1b9701ac5952267d7ee38cdefe3c0920cc8fa6d8a38d4d80bda6f692b6b34",
    "issuer": "University of Example",
    "recipient": "0xabcd1234walletAddress",
    "data": {
        "degree": "Bachelor of Science",
        "major": "Computer Science",
        "graduation_date": "2024-05-15"
    },
    "timestamp": "2025-02-26T08:52:31.449564",
    "revoked": true,
    "revocation_reason": "Incorrect major listed",
    "previous_token_id": null
}


/var/folders/f5/4kxhzrrd5_16jg94_dpsjq5m0000gn/T/ipykernel_56165/3759532336.py:22: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  self.timestamp = datetime.datetime.utcnow()
